# week 4
청원 주제 분류하는 FINE-TUNE TASK

In [1]:
device = 'cuda'

In [2]:
with open('../data/petitions_2019-01.txt', 'r') as f:
    corpus = f.readlines()

In [3]:
json_list = [eval(json.strip()) for json in corpus]

In [4]:
corpus = [(json['content'], json['category']) for json in json_list]

In [5]:
c = corpus

# base line

# data preprocess

In [6]:
import re
import mecab
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim

import torchtext
import sys
sys.path.append('../source')
from dataset import *
from txt_cleaner.clean.master import MasterCleaner
from txt_cleaner.utils import *
from torch8text.data import Vocab, Field, LabelField
from torch.utils.data import Dataset, DataLoader


In [8]:
config = read_yaml('../config.yaml')
petition_ds = PetitionDataset_finetune(config)
petition_ds = petition_ds(c)

{'MINIMUM_SPACE_COUNT': 2, 'TOKEN_MAX_LEN': 40, 'CHR_MAX_LEN': 4, 'BATCH_SIZE': 64, 'TOKEN_MIN_FREQ': 5, 'CHR_MIN_FREQ': 1, 'IS_FINETUNE': True}


In [9]:
dl = DataLoader(petition_ds, batch_size=4, collate_fn=pad_collate_finetune)

In [12]:
for _ in dl:
    print(_.src_chr.shape)
    print(_.trg)
    break

torch.Size([4, 40, 4])
tensor([16,  5,  2,  2])


In [13]:
INPUT_DIM = len(petition_ds.chr_field.vocab.stoi_dict)
OUTPUT_DIM = len(petition_ds.label_field.vocab.stoi_dict)
N_LAYERS = 2
HID_DIM = 512
EMBEDDING_DIM = 1024

In [14]:
class simpleGRU_model(nn.Module):
    def __init__(self, input_dim, embedding_dim, n_layers, hid_dim, output_dim):
        super(simpleGRU_model, self).__init__()
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hid_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hid_dim, output_dim)
        
    def forward(self, x):
        bs = x.shape[0]
        output = self.embedding(x)
        output = output.reshape(bs, -1, self.embedding_dim)
        _, output = self.gru(output)
        output = self.fc(output[-1, :, :])
        return output

In [15]:
simple_gru = simpleGRU_model(INPUT_DIM, EMBEDDING_DIM, N_LAYERS, HID_DIM, OUTPUT_DIM)

In [16]:
petition_ds.label_field.vocab.stoi_dict

{'외교/통일/국방': 0,
 '교통/건축/국토': 1,
 '일자리': 2,
 '육아/교육': 3,
 '행정': 4,
 '농산어촌': 5,
 '경제민주화': 6,
 '정치개혁': 7,
 '기타': 8,
 '보건복지': 9,
 '문화/예술/체육/언론': 10,
 '안전/환경': 11,
 '인권/성평등': 12,
 '저출산/고령화대책': 13,
 '반려동물': 14,
 '미래': 15,
 '성장동력': 16}

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(simple_gru.parameters(), lr = config['TRAIN']['LR'])

In [18]:
for _ in dl:
    output = simple_gru(_.src_chr)
    loss = criterion(output, _.trg)
    loss.backward()
    optimizer.step()

tensor(2.6471, grad_fn=<NllLossBackward>)
tensor(4.0812, grad_fn=<NllLossBackward>)
tensor(2.7391, grad_fn=<NllLossBackward>)
tensor(3.4358, grad_fn=<NllLossBackward>)
tensor(3.4198, grad_fn=<NllLossBackward>)
tensor(2.8955, grad_fn=<NllLossBackward>)
tensor(3.2312, grad_fn=<NllLossBackward>)
tensor(3.1784, grad_fn=<NllLossBackward>)
tensor(2.5244, grad_fn=<NllLossBackward>)
tensor(2.0468, grad_fn=<NllLossBackward>)
tensor(3.9767, grad_fn=<NllLossBackward>)
tensor(4.4577, grad_fn=<NllLossBackward>)
tensor(3.1601, grad_fn=<NllLossBackward>)
tensor(2.8546, grad_fn=<NllLossBackward>)
tensor(2.0003, grad_fn=<NllLossBackward>)
tensor(2.1039, grad_fn=<NllLossBackward>)
tensor(2.5347, grad_fn=<NllLossBackward>)
tensor(4.1198, grad_fn=<NllLossBackward>)
tensor(2.5586, grad_fn=<NllLossBackward>)
tensor(3.3215, grad_fn=<NllLossBackward>)
tensor(4.6038, grad_fn=<NllLossBackward>)
tensor(3.3591, grad_fn=<NllLossBackward>)
tensor(2.2038, grad_fn=<NllLossBackward>)
tensor(2.4930, grad_fn=<NllLossBac

KeyboardInterrupt: 